# `TabularPandas` and missing value errors

When processing new data using an pre-existing instance of `TabularPandas` I've encountered the following error:

AssertionError: nan values in `cont_prop` but not in setup training set

`cont_prop` is a column with a missing value in this example below.

The origin of this problem is that no missing values were seen for that column during the setup phase of the `TabularPandas` object where the transforms were applied.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from fastai2.tabular.all import *
from fastai2 import metrics

## Test on a simple data frame

Let's generate `df` to create the initial `TabularPandas` object (`to`) for, which is usually then later used for the tabular learner. Let's also create a `df_test` which will be the unseen data on which we will want to use `to.new(df_test)`.

In [43]:
def get_simple_df(n):
    return pd.DataFrame({
        "cat_prop": np.random.choice(["a", "b", "c"], size=n),
        "cont_prop": np.random.uniform(size=n)
    })

n = 10
n_test = 5

df = get_simple_df(n)
df = df.append(pd.DataFrame({"cat_prop":["b", None], "cont_prop": [np.nan, .6]}),
               ignore_index=True)
df_test = get_simple_df(n_test)
df_test = df_test.append(pd.DataFrame({"cat_prop":["a", None], "cont_prop": [np.nan, .5]}),
                         ignore_index=True)

df, df_test

(   cat_prop  cont_prop
 0         a   0.266686
 1         c   0.151567
 2         c   0.166479
 3         a   0.034086
 4         b   0.700573
 5         c   0.862857
 6         c   0.307003
 7         a   0.770959
 8         b   0.873119
 9         a   0.286313
 10        b        NaN
 11     None   0.600000,
   cat_prop  cont_prop
 0        a   0.933491
 1        c   0.331430
 2        b   0.786464
 3        a   0.895200
 4        b   0.683861
 5        a        NaN
 6     None   0.500000)

In [44]:
cat_names = ["cat_prop"]
cont_names = ["cont_prop"]

The assertion error due to a missing value will be caused if the validation or test set have nans in columns where the train set did not have any. So let's exclude row 10 from the training set.

In [45]:
dep_var = "y"
df[dep_var] = np.linspace(-1,1,len(df))**2
val_col = "is_valid"
df[val_col] = True
df.loc[4:,val_col] = False
# causing an error by excluding the sole null value for that column 
# from the setup, if not patched with, e.g., an additional row
df.loc[10,val_col] = True  
df

,cat_prop,cont_prop,y,is_valid
0,a,0.266686,1.000000,True
1,c,0.151567,0.669421,True
2,c,0.166479,0.404959,True
3,a,0.034086,0.206612,True
4,b,0.700573,0.074380,False
5,c,0.862857,0.008264,False
6,c,0.307003,0.008264,False
7,a,0.770959,0.074380,False
8,b,0.873119,0.206612,False
9,a,0.286313,0.404959,False


What `FillMissing.setups` will see

In [46]:
pd.isnull(df.loc[df[val_col]==False]).any()

cat_prop      True
cont_prop    False
y            False
is_valid     False
dtype: bool

So far so good. `cont_prop` does not appear to have nan values in the training set.

To fix this problem, without having to modify any of the fastai2 code, let's add a single row setting the column(s), where we expect to find nans in the future, to nan.

Skip this next two cells if you want to produce the assertion error for nan values

In [7]:
row = df.loc[df[val_col]==False,:].sample(n=1)
row["cont_prop"] = np.nan
row

,cat_prop,cont_prop,y,is_valid
8,c,NaN,0.206612,False


Now append this row to `df`

In [8]:
df = df.append(row, ignore_index=True)

In [47]:
df

,cat_prop,cont_prop,y,is_valid
0,a,0.266686,1.000000,True
1,c,0.151567,0.669421,True
2,c,0.166479,0.404959,True
3,a,0.034086,0.206612,True
4,b,0.700573,0.074380,False
5,c,0.862857,0.008264,False
6,c,0.307003,0.008264,False
7,a,0.770959,0.074380,False
8,b,0.873119,0.206612,False
9,a,0.286313,0.404959,False


As you can see there is now a row 12 whose `cont_prop` value is nan and belongs to the training set (`is_valid == False`)

In [48]:
pd.isnull(df.loc[df[val_col]==False]).any()

cat_prop      True
cont_prop    False
y            False
is_valid     False
dtype: bool

The `FillMissing.setups` sanity check also shows that the column of the training set now contains nan

In [49]:
procs = [Categorify, FillMissing, Normalize]
splits = ColSplitter(col=val_col)(df)

In [50]:
df["cat_prop"] = pd.Categorical(df["cat_prop"])

In [51]:
to = TabularPandas(df, procs=procs, cat_names=cat_names, cont_names=cont_names,
                   y_names=dep_var, splits=splits)

AssertionError: nan values in `cont_prop` but not in setup training set

In [14]:
to.na_dict

{'cont_prop': 0.6269944329852075}

In [15]:
to_val = to.new(df)

In [16]:
to_val.process()

In [17]:
to_test = to.new(df_test)

In [18]:
to_test.process()

## Test on house prices data frames

An actual data set where the above problem occurs is the house sale prices one can find on kaggle. So let's apply the above fix

In [19]:
df = pd.read_csv("../data/train.csv")
df_test = pd.read_csv("../data/test.csv")

In [20]:
procs = [Categorify, FillMissing, Normalize]
valid_pct = .2
val_col = "is_valid"
df[val_col] = np.random.choice([True, False], size=len(df), p=[1-valid_pct, valid_pct])

In [21]:
dep_var = "SalePrice"

In [22]:
cont_names = ['LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF2', 'BsmtUnfSF',
              'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageYrBlt', 'GarageArea',
              'WoodDeckSF', 'OpenPorchSF', 'LotFrontage', 'BsmtFinSF1', 'LowQualFinSF',]

In [23]:
cat_names = ['Id', 'MSSubClass', 'MSZoning', 'Street',
             'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
             'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
             'HouseStyle', 'OverallQual', 'OverallCond', 
             'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
             'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
             'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
             'BsmtFinType2', 'Heating',
             'HeatingQC', 'CentralAir', 'Electrical', 
             'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
             'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
             'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
             'GarageFinish', 'GarageCars', 'GarageQual',
             'GarageCond', 'PavedDrive',
             'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC',
             'Fence', 'MiscFeature', 'MiscVal', 'MoSold', 'SaleType',
             'SaleCondition']

In case we only want to use a subset of the columns for testing

In [24]:
df = df.loc[:, cont_names+cat_names+[val_col, dep_var]]
df_test = df_test.loc[:, cont_names+cat_names]

In [25]:
def display_all(df):
    "Extended displaying of a data frame"
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000):
        display(df)

In [26]:
pd.isnull(df).any()[pd.isnull(df).any()]

MasVnrArea      True
GarageYrBlt     True
LotFrontage     True
Alley           True
MasVnrType      True
BsmtQual        True
BsmtCond        True
BsmtExposure    True
BsmtFinType1    True
BsmtFinType2    True
Electrical      True
FireplaceQu     True
GarageType      True
GarageFinish    True
GarageQual      True
GarageCond      True
PoolQC          True
Fence           True
MiscFeature     True
dtype: bool

Continuous columns with nan values in the training, validation and test

In [27]:
na_cont_cols_train = [col for col in cont_names if df.loc[df[val_col]==False, col].hasnans]
na_cont_cols_train

['MasVnrArea', 'GarageYrBlt', 'LotFrontage']

In [28]:
na_cont_cols_valid = [col for col in cont_names if df.loc[df[val_col]==True, col].hasnans]
na_cont_cols_valid

['MasVnrArea', 'GarageYrBlt', 'LotFrontage']

In [29]:
na_cont_cols_test = [col for col in cont_names if df_test[col].hasnans]
na_cont_cols_test

['MasVnrArea',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'GarageYrBlt',
 'GarageArea',
 'LotFrontage',
 'BsmtFinSF1']

Merging the three lists above

In [30]:
na_cont_cols = list(set(na_cont_cols_train + na_cont_cols_valid + na_cont_cols_test))
na_cont_cols

['LotFrontage',
 'GarageArea',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'BsmtFinSF2',
 'GarageYrBlt',
 'BsmtFinSF1',
 'MasVnrArea']

Select a random row to do the fix

In [31]:
row = df.loc[df["is_valid"]==False,:].sample(n=1).copy(); display_all(row.T)

,953
LotArea,11075
YearBuilt,1969
YearRemodAdd,1969
MasVnrArea,232
BsmtFinSF2,193
BsmtUnfSF,29
TotalBsmtSF,784
1stFlrSF,1168
2ndFlrSF,800
GrLivArea,1968


Setting the column values to nan

In [32]:
row[na_cont_cols] = np.nan
display_all(row.T)

,953
LotArea,11075
YearBuilt,1969
YearRemodAdd,1969
MasVnrArea,NaN
BsmtFinSF2,NaN
BsmtUnfSF,NaN
TotalBsmtSF,NaN
1stFlrSF,1168
2ndFlrSF,800
GrLivArea,1968


`df`'s nan status before appending the row

In [33]:
pd.isnull(df.loc[df[val_col]==False, na_cont_cols]).any()

LotFrontage     True
GarageArea     False
BsmtUnfSF      False
TotalBsmtSF    False
BsmtFinSF2     False
GarageYrBlt     True
BsmtFinSF1     False
MasVnrArea      True
dtype: bool

Creating the patched dataframe `df_p`

In [34]:
print(len(df))
df_p = df.append(row, ignore_index=True)
print(len(df_p))

1460
1461


`df`'s nan status after appending the row

In [35]:
pd.isnull(df_p.loc[df_p[val_col]==False, na_cont_cols]).any()

LotFrontage    True
GarageArea     True
BsmtUnfSF      True
TotalBsmtSF    True
BsmtFinSF2     True
GarageYrBlt    True
BsmtFinSF1     True
MasVnrArea     True
dtype: bool

Generating `to`

In [36]:
dep_var = "SalePrice"
splits = ColSplitter(col=val_col)(df_p)
to = TabularPandas(df_p, procs=procs, cat_names=list(cat_names), 
                   cont_names=list(cont_names),
                   y_names=dep_var, splits=splits)

In [37]:
to.na_dict

{'MasVnrArea': 0.0,
 'BsmtFinSF2': 0.0,
 'BsmtUnfSF': 401.5,
 'TotalBsmtSF': 1011.0,
 'GarageYrBlt': 1979.0,
 'GarageArea': 476.0,
 'LotFrontage': 70.0,
 'BsmtFinSF1': 439.5}

In [39]:
to_val = to.new(df)

In [40]:
to_val.process()

In [41]:
to_test = to.new(df_test)

In [42]:
to_test.process()